In [1]:
using Plots;plotly()

Plots.PlotlyBackend()

In [2]:
using DataFrames;using CSV

In [3]:
using Optim
using Zygote

In [4]:
using LinearAlgebra
ccall((:openblas_get_num_threads64_, Base.libblas_name),Cint,())
# LinearAlgebra.BLAS.set_num_threads(8)

4

In [5]:
using Base.Threads;nthreads()

1

In [6]:
using Meteor.Utility
using Meteor.ExactDiag

In [7]:
function thermal_state(dims,nbar)
    c::Array{Complex{Float64}}=zeros(dims,dims)
    if nbar==0
        c[1,1]=1
    else
        for i in 1:dims
           c[i,i]= exp(-i*log(1/nbar+1))
        end
    end
    c=c/tr(c)
    return c        
end

function fock_state(dims,n)
    c::Matrix{ComplexF64}=zeros(dims,dims)
    if n<dims
        c[n+1,n+1]=1
    end
    return c
end

function eye(n)
    k::Array{Complex{Float64}}=zeros(n,n)
    for i in 1:n
        k[i,i]=1
    end
    return k    
end

σ₊=[0 0;1 0]
σ₋=[0 1;0 0]
σ_x=[0 1;1 0]
σ_y=[0 -1im;1im 0]
σ_z=[1 0;0 -1]

function a₋(n)      #n为维数
    s::Array{Complex{Float64}}=zeros(n,n)
    for i=1:n-1
        s[i,i+1]=sqrt(i)
    end
    return s
end

a₊(n)=adjoint(a₋(n));

dim=10
ν=1
η=0.1
γ=0.1*ν
dot_num=30
initial_state=kron([1.0 0;0 0],thermal_state(dim,1))
#########密度矩阵向量化，超算子维数变成n^2*n^2维,态矩阵厄米，只用存一半？意义不大
function dm2vec(dm)#态向量化
#julia里是按列reshape的,reshape成列向量
    return reshape(dm,:,1)
end

function vec2dm(vec)
    return reshape(vec,2*dim,:)
end

function opconv(op)#算子变大,这里算子写的是上文中边带冷却的函数
    x::Matrix{ComplexF64}=zeros(4*dim^2,4*dim^2)
    for i in 1:4*dim^2
        a::Matrix{ComplexF64}=zeros(dim*2,dim*2)
        a[i]=1
        x[:,i]=reshape(op(a),:,1)
    end
    return x
end

I=eye(dim*2)
n=kron(eye(2),a₊(dim)*a₋(dim))
en=kron([0 0;0 1],eye(dim))
a=a₊(dim)+a₋(dim)
a2=kron([0 0;1 0],exp(im*η*a))
a3=kron([0 1;0 0],exp(-im*η*a))
a4=a2+a3


function diss()
    dtheta = 0.01
    x=zeros((2*dim)^2,(2*dim)^2)
    for theta in -1:dtheta:1
        c = dtheta * 0.75 * (1 + theta^2)
        jump = sqrt(c / 2) * exp(im*η*theta*a)
        aup=kron([0 0;1 0],jump')
        adown=kron([0 1;0 0],jump)
        x=x+kron(transpose(aup),adown)-0.5*kron(I,aup*adown)-0.5*kron(transpose(aup*adown),I)
    end
    return x
end

a5=diss() 
# a5=kron(transpose(kron([0 0;1 0],eye(7))),kron([0 1;0 0],eye(7)))

# A*ρ*B形式，ρ(d*d)向量化之后，A*X*B的作用效果，变为矩阵kron(tranpose(B),A),,求单位相应ρ[i,j],,A第i列的元素放在第j列再去乘B第j行，
# A的i列乘B的j行，，，A[:,i]*B[j,:],,密度矩阵按列向量化，A*X*B变换后矩阵的第d*(j-1)+i列是reshape(A[:,i]*B[j,:],:,1),,

function sideband_Lindbladian_vec(Ω,Δ)
    unitary::Matrix{ComplexF64}=kron(I,-im*(ν*n-Δ*en+Ω/2*a4))+kron(transpose(im*((ν*n-Δ*en+Ω/2*a4))),I)
    
    dissipation=a5
     
    return unitary+γ*dissipation
end

n=kron(eye(2),a₊(dim)*a₋(dim));
instate=dm2vec(initial_state);


In [10]:
function feval(x)
    f=exp(400*sideband_Lindbladian_vec(x[1],x[2]))*instate
    return real(tr(n*vec2dm(f)))
end        
res0=optimize(feval,[0.4 -0.9],LBFGS())

 * Status: success

 * Candidate solution
    Final objective value:     8.713101e-03

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 2.39e-08 ≰ 0.0e+00
    |x - x'|/|x'|          = 2.68e-08 ≰ 0.0e+00
    |f(x) - f(x')|         = 1.85e-15 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 2.13e-13 ≰ 0.0e+00
    |g(x)|                 = 1.52e-10 ≤ 1.0e-08

 * Work counters
    Seconds run:   12  (vs limit Inf)
    Iterations:    7
    f(x) calls:    24
    ∇f(x) calls:   24


In [11]:
Optim.minimizer(res0)

1×2 Array{Float64,2}:
 0.459097  -0.8905

In [12]:
Optim.minimum(res0)

0.008713101227767105

In [34]:
function side_band_cooling_model(d, Delta, nu, eta, omega, gamma)
    ps = nlevel(["g", "e"])
    pb = boson(d=d)
    
    xop = pb["a"] + pb["adag"]
    exp_xop = exp(im*eta*xop)
    
    model = build_model([ps, pb], isunitary=false)

    add_unitary!(model, (1,), ("e<-e",), coeff=-Delta)
    add_unitary!(model, (2,), ("n",), coeff=nu)
    add_unitary!(model, (1, 2), ("e<-g", exp_xop), coeff=omega/2)
    add_unitary!(model, (1, 2), ("g<-e", exp_xop'), coeff=omega/2)
    
    dtheta = 0.01
    for theta in -1:dtheta:1
        c = dtheta * 0.75 * (1 + theta^2)
        jump = sqrt(c / 2) * exp(im*eta*theta*xop)
        add_dissipation!(model, (1, 2), ("g<-e", jump), coeff=gamma/2)
    end
# add_dissipation!(model, (1,), ("g<-e",), coeff=gamma/2)
    
    add_observer!(model, (1,), ("e<-e",), name="e")
    add_observer!(model, (2,), ("n",), name="n")
    
    model.state=kron(thermal_state(d,1),[1.0 0;0 0])
    return model
end


# explicit build the matrix for the Lindblad operator
function side_band_cooling_b(d, Delta, nu, eta, omega, gamma, t)
    model = side_band_cooling_model(d, Delta, nu, eta, omega, gamma)
    ham, observer, state = model.h, model.observer, model.state
    ham = matrix(immrep(ham))
    L = size(state, 1)
    observer = immrep(observer)
    obs = measure(observer, state)
    obs = Observables(obs)
    
    state = reshape(state, L * L)
    mt = t/100
    nt = round(Int, t / mt)
    for i in 1:nt
        state = evolve(ham, mt, state, ishermitian=false, driver=:krylov)
        append!(obs, measure(observer, reshape(state, L, L)))
    end
    return todict(obs)
end

function side_band_cooling_c( Delta,omega, t)
    model = side_band_cooling_model(dim, Delta, ν, η, omega, γ)
    ham, observer, state = model.h, model.observer, model.state
    ham = matrix(immrep(ham))
    L = size(state, 1)
    observer = immrep(observer)
    obs = measure(observer, state)
    obs = Observables(obs)
    
    state = reshape(state, L * L)

    state = evolve(ham, t, state, ishermitian=false, driver=:krylov)
    append!(obs, measure(observer, reshape(state, L, L)))
    return todict(obs)
end

# compute the steady state
function steady_state(d, Delta, nu, eta, omega, gamma)
    model = side_band_cooling_model(d, Delta, nu, eta, omega, gamma)
    results = steady_state!(model)
    return todict(results.observables)
end

steady_state (generic function with 1 method)

In [35]:
LinearAlgebra.BLAS.set_num_threads(8)

In [24]:
function testplot()
    ttot=500
    num=100
    ome=[0.1 0.2 0.3 0.4 0.5]
    state=instate
    x=[[real(tr(n*vec2dm(state)))] for i in 1:5]
    for i in 1:5
        for j in 1:num
            state= exp(ttot/num*sideband_Lindbladian_vec(ome[i],-1))*state
            append!(x[i],real(tr(n*vec2dm(state))))  
        end
        state=instate
    end
    plot(ytick=0:0.2:1)
    plot!(0:ttot/num:ttot,x[1]);plot!(0:ttot/num:ttot,x[2]);plot!(0:ttot/num:ttot,x[3]);
    plot!(0:ttot/num:ttot,x[4]);plot!(0:ttot/num:ttot,x[5])
end 
testplot()

<!DOCTYPE html>
 
 
 Plots.jl

In [62]:
function sweep_detuning(ttot)
    initial_x=[0.2]
    sweep_num=100
    start=-1.1
    stop=-0.6
    d=[]
    sx=[]
    sn=[]
    for i in 1:sweep_num
        print(i," ")
        m=start+(i-1)*(stop-start)/sweep_num
        function feval(x)
            f=exp(ttot*sideband_Lindbladian_vec(x[1],m))*instate
            return real(tr(n*vec2dm(f)))
        end        
        
#         a=optimize(feval,[0.0],[100.0],initial_x,Fminbox(LBFGS()))
        a=optimize(feval,initial_x,LBFGS())
        initial_x=Optim.minimizer(a) 
        append!(sx,initial_x[1])   
        append!(sn,Optim.minimum(a)[1])
        append!(d,m)
    end
    return sx,sn,d
end


sweep_detuning (generic function with 1 method)

In [63]:
res1=sweep_detuning(100);df1=DataFrame(delta=res1[3],omega=res1[1],n=res1[2]);CSV.write("sweep_detuning_100.csv", df1);
res2=sweep_detuning(250);df2=DataFrame(delta=res2[3],omega=res2[1],n=res2[2]);CSV.write("sweep_detuning_250.csv", df2);
res3=sweep_detuning(400);df3=DataFrame(delta=res3[3],omega=res3[1],n=res3[2]);CSV.write("sweep_detuning_400.csv", df3);


1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 

In [64]:
data100=CSV.read("sweep_detuning_100.csv", DataFrame);
data250=CSV.read("sweep_detuning_250.csv", DataFrame);
data400=CSV.read("sweep_detuning_400.csv", DataFrame);
plot(xticks = -1.1:0.05:-0.65, yticks = 0:0.2:1.2,xlims = (-1.1, -0.6), ylims = (0, 1.3))
plot!(data100.delta, data100.n, label= "n100",xlabel="Δ/ν",color=:green)
plot!(data100.delta, data100.omega, label= "omega100",color=:green)
plot!(data250.delta, data250.n, label= "n250",color=:red)
plot!(data250.delta, data250.omega, label= "omega250",color=:red)
plot!(data400.delta, data400.n, label= "n400",color=:brown)
plot!(data400.delta, data400.omega, label= "omega400",color=:brown)

<!DOCTYPE html>
 
 
 Plots.jl

In [19]:
#3维图扫一下
LinearAlgebra.BLAS.set_num_threads(1)
function nt()
    ttot=250
    space=0.01
    s_delta=-1.1:space: -0.6
    s_omega=0.3:space:0.8
    n = zeros(length(s_delta),length(s_omega))
    e = zeros(length(s_delta),length(s_omega))
    datane=DataFrame(delta=s_delta,omega=s_omega)
    @threads for omega in s_omega
        for delta in s_delta
            n[convert(Int64,round((delta-s_delta[1])/space+1)),convert(Int64,round((omega-s_omega[1])/space+1))]=side_band_cooling_c(delta,omega, ttot)["n[2]"][2]
            e[convert(Int64,round((delta-s_delta[1])/space+1)),convert(Int64,round((omega-s_omega[1])/space+1))]=side_band_cooling_c(delta,omega, ttot)["e[1]"][2]
        end
        print(omega," ")
    end
    datane=hcat(datane,DataFrame(hcat(n,e),:auto))
    CSV.write("delta_omega_sweep.csv", datane)
    return n,e
end


nt (generic function with 1 method)

In [21]:
# @time omde=nt()

In [22]:
data3d=CSV.read("delta_omega_sweep.csv", DataFrame)
nesurf=Array(data3d[:,Between(:x1, end)])
row=convert(Int64,size(nesurf)[2]/2)
nsurf=nesurf[:,1:row];
esurf=nesurf[:,row+1:end];
surface(data3d.omega,data3d.delta,nsurf,xlabel="omega",ylabel="Δ",zlabel="n_t",xticks=0.3:0.1:0.8,yticks=-1.1:0.1:-0.6,zticks=0:0.1:0.9)

<!DOCTYPE html>
 
 
 Plots.jl

In [49]:
surface([1,2,3],[4,5,6],[1 2 3;4 5 6;7 8 9])

<!DOCTYPE html>
 
 
 Plots.jl

In [48]:
#####测试对比使用
function plotme()
#     de=-0.85
#     ome=0.5142359
    num=100
    t=1500
#     state=instate
#     x=[real(tr(n*vec2dm(state)))]
#     norm=[real(tr(kron([1 0;0 0],eye(dim))*vec2dm(state))+tr(kron([0 0;0 1],eye(dim))*vec2dm(state)))]
#     for i in 1:num
#         state= exp(t/num*sideband_Lindbladian_vec(ome,de))*state
#         append!(x,real(tr(n*vec2dm(state))))  
#         append!(norm,real(tr(kron([1 0;0 0],eye(dim))*vec2dm(state))+tr(kron([0 0;0 1],eye(dim))*vec2dm(state))))
#     end
#     plot(0:t/num:t,x)
#     plot!(norm)
    
    plot(0:t/num:t,side_band_cooling_b(dim,-1,1,0.1,0.3334303,0.1,t)["n[2]"],ytick=0:0.2:1)
    plot!(0:t/num:t,side_band_cooling_b(dim,-0.845,1,0.1,0.545147,0.1,t)["n[2]"])
end
evo1=DataFrame(t=0:15:1500,evo=side_band_cooling_b(dim,-1,1,0.1,0.3334303,0.1,1500)["n[2]"]);CSV.write("250_-1.csv", evo1);
evo2=DataFrame(t=0:15:1500,evo=side_band_cooling_b(dim,-0.845,1,0.1,0.545147,0.1,1500)["n[2]"]);CSV.write("250_-0.845.csv", evo2);
readevo1=CSV.read("250_-1.csv", DataFrame);
readevo2=CSV.read("250_-0.845.csv", DataFrame);
plot(readevo1.t,readevo1.evo)
plot!(readevo2.t,readevo2.evo)

<!DOCTYPE html>
 
 
 Plots.jl

In [35]:
steady_state(dim, -1, ν, η, 0.5142359, γ)["n[2]"][1]

0.01947992449492013

In [11]:
LinearAlgebra.BLAS.set_num_threads(1)
function sweep_steady_state(result)
    sn=zeros(length(result.delta))
    sin=zeros(length(result.delta))
    @threads for i in 1:length(result.delta)
        print(i," ")
        a=steady_state(dim, result.delta[i], ν, η, result.omega[i], γ)
        sn[i]=a["n[2]"][1]
        sin[i]=a["e[1]"][1]
    end
    return sn,sin
end

sweep_steady_state (generic function with 1 method)

In [9]:
resu=CSV.read("sweep_detuning_250.csv", DataFrame);

In [12]:
s=sweep_steady_state(resu)

65 77 27 1 40 53 89 14 78 90 66 54 91 79 67 41 55 92 80 2 28 56 81 68 93 42 15 82 69 57 94 83 3 70 95 58 84 43 29 96 71 85 97 59 44 72 98 86 99 73 60 87 4 100 16 45 30 61 88 74 75 46 62 5 76 47 63 31 48 64 6 49 17 32 50 7 51 33 52 18 8 34 9 35 10 19 36 37 11 38 20 12 39 13 21 22 23 24 25 26 

([0.03816292899350905, 0.03561343553770063, 0.0331279805739385, 0.030711819560818865, 0.028370870765996106, 0.026111385169486002, 0.023939255935693435, 0.021861092590016014, 0.019883068559784736, 0.018011299202178015  …  0.049337496774241985, 0.05099953066061422, 0.05270628088269101, 0.054458820607423956, 0.05625825447518584, 0.05810571944536219, 0.06000238617761344, 0.06194946033366241, 0.0639481844596396, 0.06599983901382671], [0.05362349780308537, 0.05198074162205711, 0.050314459248421345, 0.04862779765958632, 0.046925223598973644, 0.04521231561167849, 0.0434949900480171, 0.04178130125544571, 0.04008006375756235, 0.03840173221570593  …  0.19959013409996107, 0.2026849367683512, 0.2057859534721541, 0.20889290281981118, 0.2120054955843409, 0.21512343493011205, 0.21824641631555125, 0.22137412767634848, 0.22450624934924343, 0.22764245421410795])

In [13]:
ss=DataFrame(delta=resu.delta,omega=resu.omega,nbose=s[1],pe=s[2]);CSV.write("steady250.csv", ss);

In [14]:
data_steady250=CSV.read("steady250.csv", DataFrame);
data_250=CSV.read("sweep_detuning_250.csv", DataFrame);
plot(title="ttot=250 steady n")
plot!(data_250.delta,data_250.omega,label= "omega250",color=:green)
plot!(data_250.delta,data_250.n,label= "n250",color=:red)
plot!(data_250.delta,data_steady250.nbose,label= "steady_n250",color=:brown)
plot!(data_250.delta,data_steady250.pe,label= "steady_e_femi250",color=:black)

<!DOCTYPE html>
 
 
 Plots.jl